#### Introducción al aprendizaje profundo
#### Giovanni Gamaliel López Padilla
##### Tarea 01

###### Versiones utilizadas
```bash
numpy == 1.22.3
tabulate == 0.8.9
```

In [1]:
def obtain_all_params() -> dict:
    """
    Funcion que reune los parámtros de la función y el gradiente. Devuelve estos dos parámetros en forma de diccionarios.
    """
    params = {
        "models": ["SGD",
                   "NAG",
                   "ADAM",
                   "ADADELTA"],
        "max iteration": 100,
        "n": 100,
        "sigma": 1,
        "epsilon": 0.01
    }

    # parámetros del algoritmo
    gd_params = {
        'alpha': 0.95,
        'alphaADADELTA': 0.95,
        'alphaADAM': 0.95,
        'nIter': 300,
        'batch_size': 10,
        'eta': 0.9,
        'eta1': 0.9,
        'eta2': 0.999
    }
    return params, gd_params

In [2]:
from numpy import exp, ones, mean, ones_like, array
from tabulate import tabulate


def print_results(params: dict, results: array) -> None:
    table = []
    for model_name in params["models"]:
        time = results[model_name]["time"]
        error = results[model_name]["error"]
        table += [[model_name, time, error]]
    print(tabulate(table,
                   headers=["Model",
                            "Time",
                            "Error"]))


class function_class:
    def __init__(self) -> None:
        pass

    def phi(self, y: array, mu: array, sigma: array) -> array:
        """


        Parámetros
        -----------
        y -> Patrones a Aproximar
        mu -> Array de medias
        sigma -> Vector de Desviaciones
        Output
        -----------
        phi          : matriz de kerneles
        """
        mu_aux = mu.reshape(-1, 1)
        phi = exp(-(y-mu_aux)**2/(2*sigma**2))
        return phi

    def gradient_gaussian_mu(self, theta: None, f_params: dict) -> array:
        """
        Calcula el gradiente respecto a mu
        Parámetros
        -----------
        theta
        f_params -> lista de parametros para la funcion objetivo,
        X = f_params['X'] Variable independiente
        y = f_params['y'] Variable dependiente

        Output
        -----------
            Array gradiente
        """
        # Obtengo Parámetros
        phi = f_params['X']
        alpha = f_params['Alpha']
        n = f_params['n']
        y = f_params['y']
        mu = f_params['mu']
        alpha = alpha.reshape((-1, 1))
        mu = mu.reshape((-1, 1))
        y = y.reshape((-1, 1))
        gradient = (phi @ alpha - y) @ alpha.T * \
            (y @ ones((1, n)) - ones_like(y) @ mu.T)
        return mean(gradient, axis=0)

    def gradient_gaussian_alpha(self, theta: None, f_params: dict) -> array:
        """
        Calcula el gradiente respecto a alpha
        Parámetros
        -----------
            theta
            f_params : lista de parametros para la funcion objetivo,
                        X -> f_params['X'] Variable independiente
                        y -> f_params['y'] Variable dependiente

        Output
        -----------
            Array gradiente
        """
        # Obtengo Parámetros
        phi = f_params['X']
        y = f_params['y']
        alpha = f_params['Alpha']
        gradient = phi.T @ (phi @ alpha - y)
        return mean(gradient, axis=0)

In [3]:
from numpy import array, zeros, sqrt
from numpy.random import randint


class model_class:
    def __init__(self) -> None:
        """
        Modelo que reune los metodos de 
        + Descenso de gradiente estocástico.
        + Descenso de gradiente estoc ́astico accelerado de tipo Nesterov.
        + AdaDelta
        + ADAM
        """
        pass

    def select_method(self, method_name: str):
        if method_name == "SGD":
            self.method = self.SGD
        if method_name == "NAG":
            self.method = self.NAG
        if method_name == "ADADELTA":
            self.method = self.ADADELTA
        if method_name == "ADAM":
            self.method = self.ADAM

    def SGD(self, theta: list, grad, gd_params: dict, f_params: dict,) -> array:
        """
        Descenso de gradiente estocástico

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente

        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter = gd_params['nIter'] número de iteraciones
                        alpha = gd_params['alpha'] tamaño de paso alpha
                        batch_size = gd_params['batch_size'] tamaño de la muestra

        f_params  :   lista de parametros para la funcion objetivo,
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Output
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        (high, dim) = f_params['X'].shape
        batch_size = gd_params['batch_size']
        nIter = gd_params['nIter']
        alpha = gd_params['alpha']
        Theta = []
        for t in range(nIter):
            # Set of sampled indices
            smpIdx = randint(low=0,
                             high=high,
                             size=batch_size,
                             dtype='int32')
            # sample
            smpX = f_params['X'][smpIdx]
            smpy = f_params['y'][smpIdx]
            # parametros de la funcion objetivo
            smpf_params = {"Alpha": f_params["Alpha"],
                           "mu": f_params["mu"],
                           "n": f_params["n"],
                           'X': smpX,
                           'y': smpy}
            p = grad(theta,
                     f_params=smpf_params)
            theta = theta - alpha*p
            Theta.append(theta)
        return array(Theta)

    def NAG(self, theta: list, grad, gd_params: dict, f_params: dict,):
        """
        Descenso acelerado de Nesterov

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente
        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter = gd_params['nIter'] número de iteraciones
                        alpha = gd_params['alpha'] tamaño de paso alpha
                        eta   = gd_params['eta']  parametro de inercia (0,1]
        f_params  :   lista de parametros para la funcion objetivo,
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Output
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        nIter = gd_params['nIter']
        alpha = gd_params['alpha']
        eta = gd_params['eta']
        p = zeros(theta.shape)
        Theta = []
        for t in range(nIter):
            pre_theta = theta - 2.0*alpha*p
            g = grad(pre_theta,
                     f_params=f_params)
            p = g + eta*p
            theta = theta - alpha*p
            Theta.append(theta)
        return array(Theta)

    def ADADELTA(self, theta: list, grad, gd_params: dict, f_params: dict,):
        """
        Descenso de Gradiente Adaptable (ADADELTA)

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente
        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter    = gd_params['nIter'] número de iteraciones
                        alphaADA = gd_params['alphaADADELTA'] tamaño de paso alpha
                        eta      = gd_params['eta']  parametro adaptación del alpha
        f_params  :   lista de parametros para la funcion objetivo,
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Output
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        epsilon = 1e-8
        nIter = gd_params['nIter']
        alpha = gd_params['alphaADADELTA']
        eta = gd_params['eta']
        G = zeros(theta.shape)
        g = zeros(theta.shape)
        Theta = []
        for t in range(nIter):
            g = grad(theta,
                     f_params=f_params)
            G = eta*g**2 + (1-eta)*G
            p = 1.0/(sqrt(G)+epsilon)*g
            theta = theta - alpha * p
            Theta.append(theta)
        return array(Theta)

    def ADAM(self, theta: list, grad, gd_params: dict, f_params: dict,):
        """
        Descenso de Gradiente Adaptable con Momentum(A DAM)

        Parámetros
        -----------
        theta     :   condicion inicial
        grad      :   funcion que calcula el gradiente
        gd_params :   lista de parametros para el algoritmo de descenso,
                        nIter    = gd_params['nIter'] número de iteraciones
                        alphaADA = gd_params['alphaADAM'] tamaño de paso alpha
                        eta1     = gd_params['eta1'] factor de momentum para la direccion
                                    de descenso (0,1)
                        eta2     = gd_params['eta2'] factor de momentum para la el
                                    tamaño de paso (0,1)
        f_params  :   lista de parametros para la funcion objetivo,
                        kappa = f_params['kappa'] parametro de escala (rechazo de outliers)
                        X     = f_params['X'] Variable independiente
                        y     = f_params['y'] Variable dependiente

        Output
        -----------
        Theta     :   trayectoria de los parametros
                        Theta[-1] es el valor alcanzado en la ultima iteracion
        """
        epsilon = 1e-8
        nIter = gd_params['nIter']
        alpha = gd_params['alphaADAM']
        eta1 = gd_params['eta1']
        eta2 = gd_params['eta2']
        p = zeros(theta.shape)
        v = 0.0
        Theta = []
        eta1_t = eta1
        eta2_t = eta2
        for t in range(nIter):
            g = grad(theta,
                     f_params=f_params)
            p = eta1*p + (1.0-eta1)*g
            v = eta2*v + (1.0-eta2)*(g**2)
            theta = theta - alpha * p / (sqrt(v)+epsilon)
            eta1_t *= eta1
            eta2_t *= eta2
            Theta.append(theta)
        return array(Theta)

In [4]:
from numpy.random import uniform
from numpy.linalg import norm
from numpy import linspace
import time


def solver(models: model_class, y: list, params: dict, gd_params: dict) -> tuple:
    """
    Funcion que ejecuta un algoritmo para realizar la optimización de la función dado un diccionario de parametros

    Parámetros
    -----------------------
    models -> modelo que contiene los métodos de optimización de parámetros
    y -> patrones a aproximar
    params -> diccionario que contiene los parametros de las iteraciones
    gd_params -> diccionario que contiene los parametros del modelo
    """
    max_iteration = params["max iteration"]
    epsilon = params["epsilon"]
    sigma = params["sigma"]
    n = params["n"]
    functions = function_class()
    t_init = time.clock_gettime(0)
    # Valores Iniciales
    mu = linspace(0, 100, n)
    phi = functions.phi(y, mu, sigma)
    alpha = uniform(0, sigma, n)
    # Parámetros para el gradiente
    f_params = {
        'mu': mu,
        'X': phi,
        'y': y,
        'Alpha': alpha,
        'n': n
    }
    iteration = 0
    while iteration < max_iteration:
        # descenso para alpha
        alpha = models.method(alpha,
                              grad=functions.gradient_gaussian_alpha,
                              gd_params=gd_params,
                              f_params=f_params)[-1]
        if norm(phi @ alpha - y) < epsilon:
            break
        # descenso para mu
        mu_old = mu
        mu = models.method(mu,
                           grad=functions.gradient_gaussian_mu,
                           gd_params=gd_params,
                           f_params=f_params)[-1]
        # actualizacion
        phi = functions.phi(y, mu, sigma)
        # Criterio de parada
        if norm(mu - mu_old) < epsilon:
            break
        # Número máximo de iteraciones si no hay convergencia
        iteration += 1
    t_end = time.clock_gettime(0)
    total_time = t_end - t_init
    return phi, alpha, total_time

In [5]:
from numpy.random import uniform
results = {}
params, gd_params = obtain_all_params()
models = model_class()
y = uniform(0, 1, params["n"])
for model_name in params["models"]:
    print("Resolviendo por medio de {}".format(model_name))
    results[model_name] = {}
    models.select_method(model_name)
    phi, alpha, time_solver = solver(models, y, params, gd_params)
    error = round(((phi @ alpha - y)**2).mean(), 8)
    results[model_name]["time"] = time_solver
    results[model_name]["error"] = error
print_results(params, results)

Resolviendo por medio de SGD
Resolviendo por medio de NAG
Resolviendo por medio de ADAM
Resolviendo por medio de ADADELTA
Model         Time     Error
--------  --------  --------
SGD        8.25008  0.335291
NAG       11.3339   0.335291
ADAM      14.4546   0.335291
ADADELTA  14.6655   0.335291
